## Attribute-feature building - if you remove tot1 values which are zero you get a wrong scale which doesnt detect anomalies as it should so 0 have to be removed or handled in another way
Here I will construct an optimal data presentation to be better suited for further proceessing.   
- There will be new time attributes - minute,hour,day,month,year
- I will also remove some incomplete data
- I will also calculate daily flow in m3 by subtracting all previus values from the current one

In [1]:
import plotly.express as px
import plotly.graph_objects as go

import math
import pandas as pandas
from pandas import concat
import numpy as np
from scipy.stats import pearsonr
from statistics import mean
from numpy.random import shuffle

from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt

data = pandas.read_json("../../data/water_loss_data_set_1.json")

In [2]:
data = data[(data.timeStamp != "0000-00-00 00:00:00" ) & (data.timeStamp != "2000-01-01 00:00:00")]
data = data[(data.tot1 != 0 ) & (data.analog2 != 0)]
#  uncomment if you want day,hour ...
data["timeStamp"] = pandas.to_datetime(data["timeStamp"], format="%Y-%m-%d %H:%M")

# data_with_minutes = data.copy(deep=True)
# Normalisation of pressure data
# data["analog2"] = ((data["analog2"] - data["analog2"].min()) / (data["analog2"].max() - data["analog2"].min()))

data_249 = data[data['idflowmeter'] == "MAG8000_024905H318"]
data_248 = data[data['idflowmeter'] == "MAG8000_024805H318"]
data_249 = data_249.rename_axis('SensorID-249', axis=1)
data_248 = data_248.rename_axis('SensorID-248', axis=1)

**Grouping by timestamp, some timestamps are duplicated but their tot1, tot2, analog2 values are the same so getting the mean of that doesn't impact the result**

In [3]:
"""
To see duplicated values
data_249.sort_values(by=['timeStamp'], inplace=True)
df = data_249[data_249['timeStamp'].duplicated(keep=False)]
"""

data_249_min = data_249.groupby("timeStamp").mean()
data_248_min = data_248.groupby("timeStamp").mean()
data_249_min

SensorID-249,tot1,tot2,analog2
timeStamp,,,
2018-11-23 08:55:00,0.43,0.00,1.11
2018-11-23 08:56:00,0.89,0.00,1.11
2018-11-23 08:57:00,1.38,0.00,1.11
2018-11-23 08:58:00,1.90,0.00,1.11
2018-11-23 08:59:00,2.45,0.00,1.11
...,...,...,...
2020-04-01 08:35:00,797188.56,-293.97,0.82
2020-04-01 08:36:00,797192.50,-293.97,0.81
2020-04-01 08:37:00,797195.06,-293.97,0.81


Adding time features.....

In [4]:
def add_time_features(data_org):
    data = data_org
    data['day'] = [i.day for i in data.index]
    data['month'] = [i.month for i in data.index]
    data['year'] = [i.year for i in data.index]
    data['hour'] = [i.hour for i in data.index]
    data['minute'] = [i.minute for i in data.index]
    return data
   
data_249_min = add_time_features(data_249_min)
data_248_min = add_time_features(data_248_min)
data_249_min

SensorID-249,tot1,tot2,analog2,day,month,year,hour,minute
timeStamp,,,,,,,,
2018-11-23 08:55:00,0.43,0.00,1.11,23,11,2018,8,55
2018-11-23 08:56:00,0.89,0.00,1.11,23,11,2018,8,56
2018-11-23 08:57:00,1.38,0.00,1.11,23,11,2018,8,57
2018-11-23 08:58:00,1.90,0.00,1.11,23,11,2018,8,58
2018-11-23 08:59:00,2.45,0.00,1.11,23,11,2018,8,59
...,...,...,...,...,...,...,...,...
2020-04-01 08:35:00,797188.56,-293.97,0.82,1,4,2020,8,35
2020-04-01 08:36:00,797192.50,-293.97,0.81,1,4,2020,8,36
2020-04-01 08:37:00,797195.06,-293.97,0.81,1,4,2020,8,37


## Adding m3-minute change

In [5]:
def minute_change_tot1_tot2(data):    
    data["tot1 change"] = data.tot1.diff()
    data["tot2 change"] = data.tot2.diff()
    return data.iloc[1:]
    # first row contains Nan in the new attributes

data_249_final = minute_change_tot1_tot2(data_249_min)
data_248_final = minute_change_tot1_tot2(data_248_min)
data_249_final

SensorID-249,tot1,tot2,analog2,day,month,year,hour,minute,tot1 change,tot2 change
timeStamp,,,,,,,,,,
2018-11-23 08:56:00,0.89,0.00,1.11,23,11,2018,8,56,0.46,0.0
2018-11-23 08:57:00,1.38,0.00,1.11,23,11,2018,8,57,0.49,0.0
2018-11-23 08:58:00,1.90,0.00,1.11,23,11,2018,8,58,0.52,0.0
2018-11-23 08:59:00,2.45,0.00,1.11,23,11,2018,8,59,0.55,0.0
2018-11-23 09:00:00,3.02,0.00,1.12,23,11,2018,9,0,0.57,0.0
...,...,...,...,...,...,...,...,...,...,...
2020-04-01 08:35:00,797188.56,-293.97,0.82,1,4,2020,8,35,1.25,0.0
2020-04-01 08:36:00,797192.50,-293.97,0.81,1,4,2020,8,36,3.94,0.0
2020-04-01 08:37:00,797195.06,-293.97,0.81,1,4,2020,8,37,2.56,0.0


In [10]:
# import datetime
# data_249_final.to_csv("braila_test.csv")
# datetime.fromtimestamp(0) .to_csv("braila_test.csv")
data_249_csv = data_249_final.reset_index()
data_249_csv.index = data_249_csv["timeStamp"].astype(np.int64)// 10**9
data_249_csv = data_249_csv.rename(columns={"timeStamp": "timeStamp-dateTime"})
# datetime.datetime.fromtimestamp(data_249_final.index)
# data_249_final.index.astype(np.int64)// 10**9
data_249_csv.to_csv("braila_test.csv")
data_249_csv

SensorID-249,timeStamp-dateTime,tot1,tot2,analog2,day,month,year,hour,minute,tot1 change,tot2 change
timeStamp,,,,,,,,,,,
1542963360,2018-11-23 08:56:00,0.89,0.00,1.11,23,11,2018,8,56,0.46,0.0
1542963420,2018-11-23 08:57:00,1.38,0.00,1.11,23,11,2018,8,57,0.49,0.0
1542963480,2018-11-23 08:58:00,1.90,0.00,1.11,23,11,2018,8,58,0.52,0.0
1542963540,2018-11-23 08:59:00,2.45,0.00,1.11,23,11,2018,8,59,0.55,0.0
1542963600,2018-11-23 09:00:00,3.02,0.00,1.12,23,11,2018,9,0,0.57,0.0
...,...,...,...,...,...,...,...,...,...,...,...
1585730100,2020-04-01 08:35:00,797188.56,-293.97,0.82,1,4,2020,8,35,1.25,0.0
1585730160,2020-04-01 08:36:00,797192.50,-293.97,0.81,1,4,2020,8,36,3.94,0.0
1585730220,2020-04-01 08:37:00,797195.06,-293.97,0.81,1,4,2020,8,37,2.56,0.0


Graphs have to be commented out/or output cleared because otherwise the files are over 50MB. The spikes seem to have a pattern they usually apear around 32- 38 minute in an hour and on average last around 4hours. This was seen from the graph and is not a product of calculation so it may not be true.

In [7]:
data_temp = data_249_final[(data_249_final["tot2 change"] < 100) & (data_249_final["tot2 change"] > -100)]
# the above code is for when 0 are not removed and you have to filter wrong data by hand
"""fig = px.line(data_249_final, x=data_249_final.index, y="tot1 change", height=600)
fig.show()"""

'fig = px.line(data_249_final, x=data_249_final.index, y="tot1 change", height=600)\nfig.show()'

In [8]:
"""fig = px.line(data_temp, x=data_temp.index, y="tot2 change", height=600)
fig.show()""";

## Determining new variable correlation

In [9]:
from scipy.stats import pearsonr, spearmanr, kendalltau

# pearson
# 249 - hour and year have correlations around 0.16, and the new attribute "tot1 change" has 0.147 others are not good
# 248 - tot1 has 0.81, tot2 -0.28, month has 0.44, year - 0.33, others have significantly lower coef....   

# spearman
# 248- tot1 is good, tot2 is -0.62, month - 0.48, year-0.22
# 249 - tot2 change has 0.53, hour 0.26, 

# kendall 
# 249 - hour - 0.21, tot1 change - 0,4, others are too small
# 248 - tot1 - 0.44, tot2 - -0.42, month-0.32, others are too small
pearson_coef_tot1_248, p_value_tot1_248 = kendalltau(data_248_final["tot2 change"], data_248_final["analog2"]) 
print("Pearson coef:", pearson_coef_tot1_248)

Pearson coef: -0.029201957392781062
